# Description

# Imports

In [1]:
import os, json, time, sys
from wildfire.Reader import Reader
import geojson
import requests
import pyproj
from pyproj import Transformer, Geod
from wildfire.Reader import Reader as WFReader
from tqdm import tqdm

# Load GEOJSON file reader

In [5]:
data_filepath = "./../Data/GeoJson_Exports/USGS_Wildland_Fire_Combined_Dataset.json"
print(f"Attempting to open '{data_filepath}'")
geojson_file = open(data_filepath,"r")
print(f"Using GeoJSON module to load sample file '{data_filepath}'")
gj_data = geojson.load(geojson_file)
geojson_file.close()
wfreader = WFReader(data_filepath)

Attempting to open './../Data/GeoJson_Exports/USGS_Wildland_Fire_Combined_Dataset.json'
Using GeoJSON module to load sample file './../Data/GeoJson_Exports/USGS_Wildland_Fire_Combined_Dataset.json'


# Get fire data into list of json objects

In [6]:
MAX_FEATURE_LOAD = 999999999
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 10000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    if feature_count >= MAX_FEATURE_LOAD:
        break
    feature = wfreader.next()
    
# print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")

Loaded 10000 features
Loaded 20000 features
Loaded 30000 features
Loaded 40000 features
Loaded 50000 features
Loaded 60000 features
Loaded 70000 features
Loaded 80000 features
Loaded 90000 features
Loaded 100000 features
Loaded 110000 features
Loaded 120000 features
Loaded 130000 features
Loaded a total of 135061 features


# For each fire calculate average ring distance from Leavenworth Kansas

In [ ]:
CITY_COORDS = (39.313015, -94.941147) #LAT, LON in EPSG:4326/WSG84

#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average

def calculate_fire_distances(geojson_data, coords):
    for i in tqdm(range(len(geojson_data)), position=0, leave=True):
        try:
            wf_feature = geojson_data[i]
            wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
            wf_year = wf_feature['attributes']['Fire_Year']
            ring_data = wf_feature['geometry']['rings'][0]
            wf_size = wf_feature['attributes']['GIS_Acres']
            wf_type = wf_feature['attributes']['Assigned_Fire_Type']
            distance = average_distance_from_place_to_fire_perimeter(coords, ring_data)
            wf_feature['distance_from_Leavenworth'] = distance
        except:
            print("Could not calculate distance for", wf_feature['attributes']['Listed_Fire_Names'].split(',')[0])

calculate_fire_distances(feature_list, CITY_COORDS)

 50%|██████████████████████████████████▌                                  | 67627/135061 [1:49:50<1:49:18, 10.28it/s]

In [ ]:
def get_fires_in_distance(geojson_data, miles):
    fires_in_distance = []
    for i in range(len(geojson_data)):
        wf_feature = geojson_data[i]
        wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
        wf_year = wf_feature['attributes']['Fire_Year']
        ring_data = wf_feature['geometry']['rings'][0]
        wf_size = wf_feature['attributes']['GIS_Acres']
        wf_type = wf_feature['attributes']['Assigned_Fire_Type']
        distance = wf_feature['distance_from_Leavenworth']
        print(distance)
        if(distance <= 0):
            fires_in_distance.append(wf_feature)
            #print(wf_name, distance)
    return fires_in_distance

city_fires = get_fires_in_distance(feature_list, 1250)
print("There are a total of", len(city_fires), "within 100 miles of leavenworth, kansas",)

# Estimate annual smoke estimate

In [ ]:
# Summation
# Volume of wood burned * volume of smoke produced by burning X volume of wood
# / distance from city
# Divide by N
# Divide by max
# Multiply by 500

# Compare estimate to AQI
AQI ranges from 0-500+, where 500+ is called 'beyond the AQI' and is extremely hazardous

# Predictive model based on fire data and smoke estimate, forecast to 2049

# Visualization 1: Histogram showing number of fires occuring every 50 file distance up to the max specified distance

# Visualization 2: Time series graph of total acres burned per year for the fires occuring in the specified distance from Leavenworth

# Visualization 3: Time series graph containing fire smoke estimate and AQI estimate for Leavenworth